In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt 
import datetime

In [2]:
from mochi import find_related_recent_trades_within_days,find_related_recent_logins_within_days
from mochi import get_multiple_feature_dicts_wihtin_days,get_transfer_result_wihtin_days
from mochi import check_nan_inf

In [20]:
def get_feature_emptyness(DataFrame,feature_list):
    feature_emptyness = {}
    feature_emptyness_list = []
    data_size = DataFrame.shape[0]
    for feature in feature_list:
        feature_emptyness[feature] = np.sum(DataFrame[feature] == -10)*1.0/data_size
        feature_emptyness_list.append((feature,feature_emptyness[feature]))
    return feature_emptyness_list

In [3]:
class Config:
    pass
config = pd.read_pickle('config.pkl')
print(dir(config))

data_path = '../../kaggleData/JD_logging/'
feature_path = '../../kaggleData/JD_logging/features/'

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'data_path', 'feature_dict', 'feature_path', 'model_features', 'result_path', 'single_module_validation_indice_set', 'trade_train_size', 'train_2_6_index']


In [4]:
login_tt = pd.read_csv(data_path+'login_tt.csv')
trade_tt = pd.read_csv(data_path+'trade_tt.csv')

login_tt['time'] = login_tt['time'].apply(lambda x : datetime.datetime.strptime(x , '%Y-%m-%d %H:%M:%S'))
trade_tt['time'] = trade_tt['time'].apply(lambda x : datetime.datetime.strptime(x , '%Y-%m-%d %H:%M:%S'))

In [6]:
#hcc_feautre = ['ip','city','device']
def build_hcc_stats_feature(recent_login_table,recent_trade_table,hcc_feature):
    login_size = len(recent_login_table)
    
    if login_size==0:
        #TODO:return -10 for all the results
        return {}
    
    result_dict = {}
    
    recent_logins = recent_login_table.sort_values(by = 'time',ascending = False)
    last_login = recent_logins.iloc[0]
    
    hcc_target = last_login[hcc_feature]
    
    result_dict[hcc_feature+'_repeat_count'] = len(recent_login_table[recent_login_table[hcc_feature] == hcc_target])
    
    result_dict[hcc_feature+'_repeat_rate']  = result_dict[hcc_feature+'_repeat_count']*1.0/login_size
    
    result_dict[hcc_feature+'_used_count'] = len(recent_login_table[hcc_feature].unique())
    
    hcc_feature_freq = recent_login_table.groupby(hcc_feature).size()
    
    hcc_feature_rank = hcc_feature_freq.rank(method = 'min').sort_values(ascending = False)
    
    result_dict[hcc_feature+'_freq_rank'] = hcc_feature_rank[hcc_target]
        
    result_dict[hcc_feature+'_most_freq_count'] = hcc_feature_freq.max()
    
    result_dict[hcc_feature+'_most_freq_rate'] = result_dict[hcc_feature+'_most_freq_count']*1.0/login_size
        
    result_dict[hcc_feature+'_max_min_dist'] =  (result_dict[hcc_feature+'_most_freq_count'] - hcc_feature_freq.min())*1.0/\
                                                 result_dict[hcc_feature+'_most_freq_count']
    return result_dict

def mult_type_hcc_feature(recent_login_table,recent_trade_table):
    hcc_feature_list = ['ip','device','city']
    result_dict = {}
    for feature in hcc_feature_list:
        temp_result = build_hcc_stats_feature(recent_login_table,recent_trade_table,feature)
        result_dict=dict(result_dict, **temp_result)
    return result_dict

In [8]:
#getting row as example for test
#lack testing
recent_trade_table=find_related_recent_trades_within_days(trade_tt.loc[89757],trade_tt,30)
recent_login_table=find_related_recent_logins_within_days(trade_tt.loc[89757],login_tt,30)
mult_type_hcc_feature(recent_login_table,recent_trade_table)

{'city_freq_rank': 2.0,
 'city_max_min_dist': 0.5,
 'city_most_freq_count': 2,
 'city_most_freq_rate': 0.66666666666666663,
 'city_repeat_count': 2,
 'city_repeat_rate': 0.6666666666666666,
 'city_used_count': 2,
 'device_freq_rank': 1.0,
 'device_max_min_dist': 0.0,
 'device_most_freq_count': 1,
 'device_most_freq_rate': 0.33333333333333331,
 'device_repeat_count': 1,
 'device_repeat_rate': 0.3333333333333333,
 'device_used_count': 3,
 'ip_freq_rank': 1.0,
 'ip_max_min_dist': 0.0,
 'ip_most_freq_count': 1,
 'ip_most_freq_rate': 0.33333333333333331,
 'ip_repeat_count': 1,
 'ip_repeat_rate': 0.3333333333333333,
 'ip_used_count': 3}

In [75]:
#abundant testing
recent_trade_table=find_related_recent_trades_within_days(trade_tt.loc[102346],trade_tt,360)
recent_login_table=find_related_recent_logins_within_days(trade_tt.loc[102346],login_tt,360)
mult_type_hcc_feature(recent_login_table,recent_trade_table)

{'city_freq_rank': 1.0,
 'city_max_min_dist': 0.0,
 'city_most_freq_count': 5,
 'city_most_freq_rate': 1.0,
 'city_repeat_count': 5,
 'city_repeat_rate': 1.0,
 'city_used_count': 1,
 'device_freq_rank': 1.0,
 'device_max_min_dist': 0.0,
 'device_most_freq_count': 5,
 'device_most_freq_rate': 1.0,
 'device_repeat_count': 5,
 'device_repeat_rate': 1.0,
 'device_used_count': 1,
 'ip_freq_rank': 1.0,
 'ip_max_min_dist': 0.0,
 'ip_most_freq_count': 5,
 'ip_most_freq_rate': 1.0,
 'ip_repeat_count': 5,
 'ip_repeat_rate': 1.0,
 'ip_used_count': 1}

In [9]:
"""
works on spark
"""
import pyspark as spark
date_range_list = [360,30,7]

conf = spark.SparkConf().setAppName('jupyter_backend').setMaster('local[15]')\
        .set('spark.executor.memory','2g')\
        .set('spark.default.parallelism','112')
sc = spark.SparkContext(conf=conf)

with sc:
    result_df = get_transfer_result_wihtin_days(trade_tt,login_tt,date_range_list,sc,mult_type_hcc_feature)

In [10]:
result_df.to_csv(data_path+'temp/feature_set_f.csv',index = False)
#result_df = pd.read_csv(data_path+'temp/feature_set_f.csv')

In [11]:
del result_df['rowkey']
del result_df['ori_id']
trade_tt = pd.concat([trade_tt,result_df],axis = 1)
print(trade_tt.shape)

feature_list = list(trade_tt.columns)
feature_list.remove('rowkey')
feature_list.remove('time')
feature_list.remove('id')
feature_list.remove('from')

trade_tt = trade_tt.fillna(-10)

(150594, 67)


In [12]:
print(feature_list)

['city_freq_rank_30', 'city_freq_rank_360', 'city_freq_rank_7', 'city_max_min_dist_30', 'city_max_min_dist_360', 'city_max_min_dist_7', 'city_most_freq_count_30', 'city_most_freq_count_360', 'city_most_freq_count_7', 'city_most_freq_rate_30', 'city_most_freq_rate_360', 'city_most_freq_rate_7', 'city_repeat_count_30', 'city_repeat_count_360', 'city_repeat_count_7', 'city_repeat_rate_30', 'city_repeat_rate_360', 'city_repeat_rate_7', 'city_used_count_30', 'city_used_count_360', 'city_used_count_7', 'device_freq_rank_30', 'device_freq_rank_360', 'device_freq_rank_7', 'device_max_min_dist_30', 'device_max_min_dist_360', 'device_max_min_dist_7', 'device_most_freq_count_30', 'device_most_freq_count_360', 'device_most_freq_count_7', 'device_most_freq_rate_30', 'device_most_freq_rate_360', 'device_most_freq_rate_7', 'device_repeat_count_30', 'device_repeat_count_360', 'device_repeat_count_7', 'device_repeat_rate_30', 'device_repeat_rate_360', 'device_repeat_rate_7', 'device_used_count_30', 'de

In [13]:
trade_tt

,rowkey,time,id,from,city_freq_rank_30,city_freq_rank_360,city_freq_rank_7,city_max_min_dist_30,city_max_min_dist_360,city_max_min_dist_7,...,ip_most_freq_rate_7,ip_repeat_count_30,ip_repeat_count_360,ip_repeat_count_7,ip_repeat_rate_30,ip_repeat_rate_360,ip_repeat_rate_7,ip_used_count_30,ip_used_count_360,ip_used_count_7
0,3,2015-01-01 00:00:41,27863,train,-10.0,-10.0,-10.0,-10.000000,-10.000000,-10.000000,...,-10.000000,-10.0,-10.0,-10.0,-10.000000,-10.000000,-10.000000,-10.0,-10.0,-10.0
1,6,2015-01-01 00:00:46,115201,train,-10.0,-10.0,-10.0,-10.000000,-10.000000,-10.000000,...,-10.000000,-10.0,-10.0,-10.0,-10.000000,-10.000000,-10.000000,-10.0,-10.0,-10.0
2,13,2015-01-01 00:01:02,143711,train,-10.0,-10.0,-10.0,-10.000000,-10.000000,-10.000000,...,-10.000000,-10.0,-10.0,-10.0,-10.000000,-10.000000,-10.000000,-10.0,-10.0,-10.0
3,32,2015-01-01 00:03:00,172699,train,-10.0,-10.0,-10.0,-10.000000,-10.000000,-10.000000,...,-10.000000,-10.0,-10.0,-10.0,-10.000000,-10.000000,-10.000000,-10.0,-10.0,-10.0
4,43,2015-01-01 00:03:52,53265,train,-10.0,-10.0,-10.0,-10.000000,-10.000000,-10.000000,...,-10.000000,-10.0,-10.0,-10.0,-10.000000,-10.000000,-10.000000,-10.0,-10.0,-10.0
5,67,2015-01-01 00:05:51,18231,train,-10.0,-10.0,-10.0,-10.000000,-10.000000,-10.000000,...,-10.000000,-10.0,-10.0,-10.0,-10.000000,-10.000000,-10.000000,-10.0,-10.0,-10.0
6,94,2015-01-01 00:10:12,147174,train,-10.0,-10.0,-10.0,-10.000000,-10.000000,-10.000000,...,-10.000000,-10.0,-10.0,-10.0,-10.000000,-10.000000,-10.000000,-10.0,-10.0,-10.0
7,98,2015-01-01 00:10:48,141674,train,-10.0,-10.0,-10.0,-10.000000,-10.000000,-10.000000,...,-10.000000,-10.0,-10.0,-10.0,-10.000000,-10.000000,-10.000000,-10.0,-10.0,-10.0
8,116,2015-01-01 00:13:16,105324,train,-10.0,-10.0,-10.0,-10.000000,-10.000000,-10.000000,...,-10.000000,-10.0,-10.0,-10.0,-10.000000,-10.000000,-10.000000,-10.0,-10.0,-10.0
9,147,2015-01-01 00:19:07,168749,train,-10.0,-10.0,-10.0,-10.000000,-10.000000,-10.000000,...,-10.000000,-10.0,-10.0,-10.0,-10.000000,-10.000000,-10.000000,-10.0,-10.0,-10.0


In [24]:
for feature in feature_list:
    pd.to_pickle(trade_tt[feature].values,feature_path+feature+'.pkl')

In [14]:
check_nan_inf(trade_tt,feature_list)

In [23]:
sorted(get_feature_emptyness(trade_tt,feature_list),key = lambda x : x[1],reverse = True)

[('city_freq_rank_7', 0.32930262825876194),
 ('city_max_min_dist_7', 0.32930262825876194),
 ('city_most_freq_count_7', 0.32930262825876194),
 ('city_most_freq_rate_7', 0.32930262825876194),
 ('city_repeat_count_7', 0.32930262825876194),
 ('city_repeat_rate_7', 0.32930262825876194),
 ('city_used_count_7', 0.32930262825876194),
 ('device_freq_rank_7', 0.32930262825876194),
 ('device_max_min_dist_7', 0.32930262825876194),
 ('device_most_freq_count_7', 0.32930262825876194),
 ('device_most_freq_rate_7', 0.32930262825876194),
 ('device_repeat_count_7', 0.32930262825876194),
 ('device_repeat_rate_7', 0.32930262825876194),
 ('device_used_count_7', 0.32930262825876194),
 ('ip_freq_rank_7', 0.32930262825876194),
 ('ip_max_min_dist_7', 0.32930262825876194),
 ('ip_most_freq_count_7', 0.32930262825876194),
 ('ip_most_freq_rate_7', 0.32930262825876194),
 ('ip_repeat_count_7', 0.32930262825876194),
 ('ip_repeat_rate_7', 0.32930262825876194),
 ('ip_used_count_7', 0.32930262825876194),
 ('city_freq_ran

In [25]:
config.feature_dict.keys()

dict_keys(['trade_and_recent_login_comparing', 'recent_login_detail', 'trade_detail_feature', 'login_trade_hist_stats', 'llc_user_habbit'])

In [26]:
config.feature_dict['hcc_user_habbit'] = feature_list

In [27]:
pd.to_pickle(config,'config.pkl')